In [1]:
import numpy as np
import tensorflow as tf
import pandas as pd

In [2]:
from tensorflow.keras.layers import Dense

In [3]:
data=pd.read_csv("C:/Users/nchar/Desktop/english_to_french_translator/eng_-french.csv")
df=pd.DataFrame(data)
print(df.info())
df.dropna(axis=0,inplace=True)
inp=df["French words/sentences"][0:10000]
inp=list(inp)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 175621 entries, 0 to 175620
Data columns (total 2 columns):
 #   Column                   Non-Null Count   Dtype 
---  ------                   --------------   ----- 
 0   English words/sentences  175621 non-null  object
 1   French words/sentences   175621 non-null  object
dtypes: object(2)
memory usage: 2.7+ MB
None


In [4]:
inp=[(word.split()) for word in inp]

In [5]:
inp

[['Salut!'],
 ['Cours', '!'],
 ['Courez', '!'],
 ['Qui', '?'],
 ['Ça', 'alors', '!'],
 ['Au', 'feu', '!'],
 ['À', "l'aide", '!'],
 ['Saute.'],
 ['Ça', 'suffit', '!'],
 ['Stop', '!'],
 ['Arrête-toi', '!'],
 ['Attends', '!'],
 ['Attendez', '!'],
 ['Poursuis.'],
 ['Continuez.'],
 ['Poursuivez.'],
 ['Bonjour', '!'],
 ['Salut', '!'],
 ['Je', 'comprends.'],
 ["J'essaye."],
 ["J'ai", 'gagné', '!'],
 ['Je', "l'ai", 'emporté', '!'],
 ['J’ai', 'gagné.'],
 ['Oh', 'non', '!'],
 ['Attaque', '!'],
 ['Attaquez', '!'],
 ['Santé', '!'],
 ['À', 'votre', 'santé', '!'],
 ['Merci', '!'],
 ['Tchin-tchin', '!'],
 ['Lève-toi.'],
 ['Va,', 'maintenant.'],
 ['Allez-y', 'maintenant.'],
 ['Vas-y', 'maintenant.'],
 ["J'ai", 'pigé', '!'],
 ['Compris', '!'],
 ['Pigé', '?'],
 ['Compris', '?'],
 ["T'as", 'capté', '?'],
 ['Monte.'],
 ['Montez.'],
 ['Serre-moi', 'dans', 'tes', 'bras', '!'],
 ['Serrez-moi', 'dans', 'vos', 'bras', '!'],
 ['Je', 'suis', 'tombée.'],
 ['Je', 'suis', 'tombé.'],
 ['Je', 'sais.'],
 ['Je', 'suis'

In [6]:
t=tf.keras.preprocessing.text.Tokenizer()
t.fit_on_texts(inp)

In [7]:
inp_encds=t.texts_to_sequences(inp)
inp_encds


[[2504],
 [1016, 2],
 [1415, 2],
 [30, 3],
 [27, 1416, 2],
 [78, 765, 2],
 [19, 1417, 2],
 [2505],
 [27, 1418, 2],
 [2506, 2],
 [1419, 2],
 [228, 2],
 [284, 2],
 [2507],
 [2508],
 [2509],
 [1017, 2],
 [766, 2],
 [1, 1420],
 [1421],
 [11, 317, 2],
 [1, 52, 606, 2],
 [144, 269],
 [1422, 451, 2],
 [2510, 2],
 [2511, 2],
 [767, 2],
 [19, 118, 767, 2],
 [285, 2],
 [2512, 2],
 [1423],
 [1424, 126],
 [2513, 126],
 [768, 126],
 [11, 1425, 2],
 [607, 2],
 [1425, 3],
 [607, 3],
 [1018, 2514, 3],
 [2515],
 [2516],
 [1019, 97, 199, 769, 2],
 [1426, 97, 286, 769, 2],
 [1, 4, 2517],
 [1, 4, 2518],
 [1, 770],
 [1, 4, 520],
 [1, 4, 771],
 [11, 399],
 [11, 179],
 [144, 452],
 [11, 2519, 400],
 [1, 89, 63],
 [27, 772],
 [1427, 2],
 [8, 9, 1428, 2],
 [2520, 2],
 [20, 773, 2521],
 [287, 2522, 2],
 [8, 1020, 12, 1021, 2],
 [6, 145, 13, 9, 1021, 2],
 [8, 2523, 2],
 [20, 255, 1429, 2],
 [1020, 12, 1021, 2],
 [288, 3],
 [318, 3],
 [2524, 200, 3],
 [285, 2],
 [70, 1430],
 [5, 53, 269],
 [5, 2525],
 [5, 289, 60

In [8]:
m=max([len(text) for text in inp_encds])
def zero_padding_and_masking(sequences,max_len):
    batch_size=len(sequences)
    padded_sequences=np.zeros((batch_size,max_len),dtype=np.int32)
    mask=np.zeros((batch_size,max_len))
    for i,seq in enumerate(sequences):
        seq_len=len(seq)
        padded_sequences[i,:seq_len]=seq
        mask[i,:seq_len]=1
    mask[mask==0]=-np.infty
    mask[mask==1]=0
    mask=tf.reshape(mask,(batch_size,1,max_len)) # here we mask shape is (batch_size,seq_len) we reshape it to (batch_size,1,seq_len) or (100000,1,10)  because 
    # the shape of look_ahead_mask will be (batch_size,seq_len,seq_len) or (10000,10,10).hence reshape the padding_mask so that during addition of both array the padding mask will added perfectly with the help of broadcastting
    mask=tf.broadcast_to(mask,(10000,10,10))
    mask=tf.expand_dims(mask,axis=1) # here we expanded mask along the dimension 1 so it will be converted to (batch_size,1,seq_len,seq_len).we do this because when we add the mask to the qk/dk or attention weights the size of attention weights is (batch_size,num_heads,seq_len,seq_len

    return padded_sequences,mask
        
        
        
    
    
inp_encds,padding_mask=zero_padding_and_masking(inp_encds,m)
inp_encds.shape
    

(10000, 10)

In [9]:
padding_mask

<tf.Tensor: shape=(10000, 1, 10, 10), dtype=float64, numpy=
array([[[[  0., -inf, -inf, ..., -inf, -inf, -inf],
         [  0., -inf, -inf, ..., -inf, -inf, -inf],
         [  0., -inf, -inf, ..., -inf, -inf, -inf],
         ...,
         [  0., -inf, -inf, ..., -inf, -inf, -inf],
         [  0., -inf, -inf, ..., -inf, -inf, -inf],
         [  0., -inf, -inf, ..., -inf, -inf, -inf]]],


       [[[  0.,   0., -inf, ..., -inf, -inf, -inf],
         [  0.,   0., -inf, ..., -inf, -inf, -inf],
         [  0.,   0., -inf, ..., -inf, -inf, -inf],
         ...,
         [  0.,   0., -inf, ..., -inf, -inf, -inf],
         [  0.,   0., -inf, ..., -inf, -inf, -inf],
         [  0.,   0., -inf, ..., -inf, -inf, -inf]]],


       [[[  0.,   0., -inf, ..., -inf, -inf, -inf],
         [  0.,   0., -inf, ..., -inf, -inf, -inf],
         [  0.,   0., -inf, ..., -inf, -inf, -inf],
         ...,
         [  0.,   0., -inf, ..., -inf, -inf, -inf],
         [  0.,   0., -inf, ..., -inf, -inf, -inf],
      

In [10]:
padding_mask.shape

TensorShape([10000, 1, 10, 10])

In [11]:
inp_encds

array([[2504,    0,    0, ...,    0,    0,    0],
       [1016,    2,    0, ...,    0,    0,    0],
       [1415,    2,    0, ...,    0,    0,    0],
       ...,
       [  38,   95,  413, ...,    0,    0,    0],
       [  38,   95, 1093, ...,    0,    0,    0],
       [  38,   95, 1094, ...,    0,    0,    0]])

In [12]:
len(t.word_index)

5525

In [13]:
embedding_dim=d_model=128
vocab_size=10000
embedding_layer=tf.keras.layers.Embedding(vocab_size,embedding_dim)
embedded_tokens=embedding_layer(inp_encds)
print(embedded_tokens.shape)

(10000, 10, 128)


In [14]:
num_heads=8
seq_len=10
dim_head=embedding_dim//num_heads
dim_head

16

In [15]:
def look_ahead_masking(seq_len,batch_size):
    mask=np.tril(np.ones((seq_len,seq_len)))
    mask[mask==0]=-np.infty
    mask[mask==1]=0
    mask=np.repeat(mask[np.newaxis,:,:],batch_size,axis=0)
    mask=tf.convert_to_tensor(mask)
    mask=tf.expand_dims(mask,axis=1)
    return mask
    

In [16]:
look_ahead_mask=look_ahead_masking(10,10000)
look_ahead_mask

<tf.Tensor: shape=(10000, 1, 10, 10), dtype=float64, numpy=
array([[[[  0., -inf, -inf, ..., -inf, -inf, -inf],
         [  0.,   0., -inf, ..., -inf, -inf, -inf],
         [  0.,   0.,   0., ..., -inf, -inf, -inf],
         ...,
         [  0.,   0.,   0., ...,   0., -inf, -inf],
         [  0.,   0.,   0., ...,   0.,   0., -inf],
         [  0.,   0.,   0., ...,   0.,   0.,   0.]]],


       [[[  0., -inf, -inf, ..., -inf, -inf, -inf],
         [  0.,   0., -inf, ..., -inf, -inf, -inf],
         [  0.,   0.,   0., ..., -inf, -inf, -inf],
         ...,
         [  0.,   0.,   0., ...,   0., -inf, -inf],
         [  0.,   0.,   0., ...,   0.,   0., -inf],
         [  0.,   0.,   0., ...,   0.,   0.,   0.]]],


       [[[  0., -inf, -inf, ..., -inf, -inf, -inf],
         [  0.,   0., -inf, ..., -inf, -inf, -inf],
         [  0.,   0.,   0., ..., -inf, -inf, -inf],
         ...,
         [  0.,   0.,   0., ...,   0., -inf, -inf],
         [  0.,   0.,   0., ...,   0.,   0., -inf],
      

In [17]:
look_ahead_mask.shape

TensorShape([10000, 1, 10, 10])

In [18]:
padding_mask

<tf.Tensor: shape=(10000, 1, 10, 10), dtype=float64, numpy=
array([[[[  0., -inf, -inf, ..., -inf, -inf, -inf],
         [  0., -inf, -inf, ..., -inf, -inf, -inf],
         [  0., -inf, -inf, ..., -inf, -inf, -inf],
         ...,
         [  0., -inf, -inf, ..., -inf, -inf, -inf],
         [  0., -inf, -inf, ..., -inf, -inf, -inf],
         [  0., -inf, -inf, ..., -inf, -inf, -inf]]],


       [[[  0.,   0., -inf, ..., -inf, -inf, -inf],
         [  0.,   0., -inf, ..., -inf, -inf, -inf],
         [  0.,   0., -inf, ..., -inf, -inf, -inf],
         ...,
         [  0.,   0., -inf, ..., -inf, -inf, -inf],
         [  0.,   0., -inf, ..., -inf, -inf, -inf],
         [  0.,   0., -inf, ..., -inf, -inf, -inf]]],


       [[[  0.,   0., -inf, ..., -inf, -inf, -inf],
         [  0.,   0., -inf, ..., -inf, -inf, -inf],
         [  0.,   0., -inf, ..., -inf, -inf, -inf],
         ...,
         [  0.,   0., -inf, ..., -inf, -inf, -inf],
         [  0.,   0., -inf, ..., -inf, -inf, -inf],
      

In [19]:
combined_mask=padding_mask+look_ahead_mask
combined_mask

<tf.Tensor: shape=(10000, 1, 10, 10), dtype=float64, numpy=
array([[[[  0., -inf, -inf, ..., -inf, -inf, -inf],
         [  0., -inf, -inf, ..., -inf, -inf, -inf],
         [  0., -inf, -inf, ..., -inf, -inf, -inf],
         ...,
         [  0., -inf, -inf, ..., -inf, -inf, -inf],
         [  0., -inf, -inf, ..., -inf, -inf, -inf],
         [  0., -inf, -inf, ..., -inf, -inf, -inf]]],


       [[[  0., -inf, -inf, ..., -inf, -inf, -inf],
         [  0.,   0., -inf, ..., -inf, -inf, -inf],
         [  0.,   0., -inf, ..., -inf, -inf, -inf],
         ...,
         [  0.,   0., -inf, ..., -inf, -inf, -inf],
         [  0.,   0., -inf, ..., -inf, -inf, -inf],
         [  0.,   0., -inf, ..., -inf, -inf, -inf]]],


       [[[  0., -inf, -inf, ..., -inf, -inf, -inf],
         [  0.,   0., -inf, ..., -inf, -inf, -inf],
         [  0.,   0., -inf, ..., -inf, -inf, -inf],
         ...,
         [  0.,   0., -inf, ..., -inf, -inf, -inf],
         [  0.,   0., -inf, ..., -inf, -inf, -inf],
      

In [20]:
class MultiHeadAttention(tf.keras.layers.Layer):
    def __init__(self,d_model:int,num_heads:int) -> None :
        super().__init__()
        self.d_model=d_model
        self.num_heads=num_heads
        self.wq=Dense(d_model)
        self.wk=Dense(d_model)
        self.wv=Dense(d_model)
        self.dense=Dense(d_model)
        self.d_head=d_model//num_heads

    
    def scaled_dot_product_attention(self,q,k,v,mask):
        a=tf.matmul(q,k,transpose_b=True)
        dk = tf.cast(tf.shape(k)[-1], tf.float32)
        attention_weights=a/tf.math.sqrt(dk)
        if mask is not None:
            attention_weights +=mask
        attention_scores=tf.nn.softmax(attention_weights)
        outputs=tf.matmul(attention_scores,v)
        return outputs,attention_scores

    
    def split_heads(self, x, batch_size,seq_len):
        x = tf.reshape(x, (batch_size,seq_len, self.num_heads, self.d_head))
        return tf.transpose(x, perm=[0, 2, 1, 3])

    
    def call(self,x,mask):
        batch_size=tf.shape(x)[0]
        seq_len=tf.shape(x)[1]
        
        q=self.wq(x)
        k=self.wk(x)
        v=self.wv(x)
        q=self.split_heads(q,batch_size,seq_len)
        k=self.split_heads(k,batch_size,seq_len)
        v=self.split_heads(v,batch_size,seq_len)
        outputs,attention_weights=self.scaled_dot_product_attention(q,k,v,mask)
        attention = tf.transpose(outputs, perm=[0, 2, 1, 3])
        attention = tf.reshape(attention, (batch_size, -1, self.d_model))
        output = self.dense(attention)
        return output

# Instantiate the MultiHeadAttention layer

# Example number of attention heads
mha_layer = MultiHeadAttention(d_model, num_heads)
# Call the layer
output = mha_layer(embedded_tokens,mask=combined_mask)
output

# Now `output` will be the result of the multi-head attention operation


<tf.Tensor: shape=(10000, 10, 128), dtype=float32, numpy=
array([[[-0.00908201, -0.00846017, -0.02830504, ..., -0.02260443,
          0.03156678,  0.02852317],
        [-0.00908201, -0.00846017, -0.02830504, ..., -0.02260443,
          0.03156678,  0.02852317],
        [-0.00908201, -0.00846017, -0.02830504, ..., -0.02260443,
          0.03156678,  0.02852317],
        ...,
        [-0.00908201, -0.00846017, -0.02830504, ..., -0.02260443,
          0.03156678,  0.02852317],
        [-0.00908201, -0.00846017, -0.02830504, ..., -0.02260443,
          0.03156678,  0.02852317],
        [-0.00908201, -0.00846017, -0.02830504, ..., -0.02260443,
          0.03156678,  0.02852317]],

       [[-0.0102041 , -0.04333679,  0.0251405 , ..., -0.0192775 ,
          0.01450991,  0.02940129],
        [-0.00262541, -0.04377333,  0.0146581 , ...,  0.00567272,
          0.02014492,  0.00498733],
        [-0.00262082, -0.04377886,  0.01465521, ...,  0.0056767 ,
          0.02015178,  0.00498697],
        .

In [21]:
mha_layer.trainable_variables

[<KerasVariable shape=(128, 128), dtype=float32, path=multi_head_attention/dense/kernel>,
 <KerasVariable shape=(128,), dtype=float32, path=multi_head_attention/dense/bias>,
 <KerasVariable shape=(128, 128), dtype=float32, path=multi_head_attention/dense_1/kernel>,
 <KerasVariable shape=(128,), dtype=float32, path=multi_head_attention/dense_1/bias>,
 <KerasVariable shape=(128, 128), dtype=float32, path=multi_head_attention/dense_2/kernel>,
 <KerasVariable shape=(128,), dtype=float32, path=multi_head_attention/dense_2/bias>,
 <KerasVariable shape=(128, 128), dtype=float32, path=multi_head_attention/dense_3/kernel>,
 <KerasVariable shape=(128,), dtype=float32, path=multi_head_attention/dense_3/bias>]

In [22]:
class Feedforward(tf.keras.layers.Layer):
    def __init__(self,d_model,dff):
        super().__init__()
        self.d_model=d_model
        self.dff=dff
        self.layer1=tf.keras.layers.Dense(dff,activation="relu")
        self.layer2=tf.keras.layers.Dense(d_model)
    def call(self,x):
        out1=self.layer1(x)
        out2=self.layer2(out1)
        return out2

In [23]:
class CrossMultiHeadAttention(tf.keras.layers.Layer):
    def __init__(self,d_model,num_heads):
        super().__init__()
        self.num_heads=num_heads
        self.d_model=d_model
        self.wq=tf.keras.layers.Dense(d_model)
        self.wk=tf.keras.layers.Dense(d_model)
        self.wv=tf.keras.layers.Dense(d_model)
        self.dense=tf.keras.layers.Dense(d_model)
        self.d_head=d_model//num_heads


    def split_heads(self,y,batch_size,seq_len):
        y=tf.reshape(y,(batch_size,seq_len,self.num_heads,self.d_head))
        y=tf.transpose(y,perm=[0,2,1,3])
        return y


    def scaled_dot_product_attention(self,q,k,v,mask):
        qk=tf.matmul(q,tf.transpose(k,perm=[0,1,3,2]))
        dk=float(tf.shape(k)[-1])
        attention_weights=qk/tf.math.sqrt(dk)
        if mask is not None:
            attention_weights +=mask
        attention_score=tf.nn.softmax(attention_weights)
        output=tf.matmul(attention_score,v)
        return output


    def call(self,x,y,mask):
        q=self.wq(y)
        k=self.wk(x)
        v=self.wv(x)
        batch_size=tf.shape(q)[0]
        seq_len=tf.shape(q)[1]
        q=self.split_heads(q,batch_size,seq_len)
        k=self.split_heads(k,batch_size,seq_len)
        v=self.split_heads(v,batch_size,seq_len)
        attention=self.scaled_dot_product_attention(q,k,v,mask)
        out=self.dense(attention)
        ouput=tf.transpose(out,perm=[0,2,1,3])
        final_output=tf.reshape(output,(batch_size,seq_len,self.d_model))
        return final_output


mha=CrossMultiHeadAttention(d_model,num_heads)
out=mha(output,embedded_tokens,mask=padding_mask)
out


        
    

<tf.Tensor: shape=(10000, 10, 128), dtype=float32, numpy=
array([[[-0.00908201, -0.00846017, -0.02830504, ..., -0.02260443,
          0.03156678,  0.02852317],
        [-0.00908201, -0.00846017, -0.02830504, ..., -0.02260443,
          0.03156678,  0.02852317],
        [-0.00908201, -0.00846017, -0.02830504, ..., -0.02260443,
          0.03156678,  0.02852317],
        ...,
        [-0.00908201, -0.00846017, -0.02830504, ..., -0.02260443,
          0.03156678,  0.02852317],
        [-0.00908201, -0.00846017, -0.02830504, ..., -0.02260443,
          0.03156678,  0.02852317],
        [-0.00908201, -0.00846017, -0.02830504, ..., -0.02260443,
          0.03156678,  0.02852317]],

       [[-0.0102041 , -0.04333679,  0.0251405 , ..., -0.0192775 ,
          0.01450991,  0.02940129],
        [-0.00262541, -0.04377333,  0.0146581 , ...,  0.00567272,
          0.02014492,  0.00498733],
        [-0.00262082, -0.04377886,  0.01465521, ...,  0.0056767 ,
          0.02015178,  0.00498697],
        .

In [24]:
class Decoder_layer(tf.keras.layers.Layer):
    def __init__(self,d_model,num_heads,dff):
        super().__init__()
        self.d_model=d_model
        self.num_heads=num_heads
        self.dff=dff
        self.attention=MultiHeadAttention(d_model=d_model,num_heads=num_heads)
        self.norm_and_add1=tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.norm_and_add2=tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.encoder_decoder_attention=CrossMultiHeadAttention(d_model=d_model,num_heads=num_heads)
        self.feed_forward=Feedforward(dff=dff,d_model=d_model)
        self.norm_and_add3=tf.keras.layers.LayerNormalization(epsilon=1e-6)
    def call(self,x,y,combined_mask,padding_mask): #x==>encoder's final query vector(output)
        attention_out=self.attention(y,mask=combined_mask)
        norm_and_add1_out=self.norm_and_add1(attention_out+y)
        encoder_decoder_attention_out=self.encoder_decoder_attention(x,y,mask=padding_mask)
        norm_and_add2_out=self.norm_and_add2(norm_and_add1_out+encoder_decoder_attention_out)
        feed_forward_out=self.feed_forward(norm_and_add2_out)
        norm_and_add3_out=self.norm_and_add3(norm_and_add2_out+feed_forward_out)
        return norm_and_add3_out

Decoder=Decoder_layer(d_model,num_heads,dff=512)
ta=Decoder(output,embedded_tokens,combined_mask,padding_mask)
ta

    

<tf.Tensor: shape=(10000, 10, 128), dtype=float32, numpy=
array([[[-0.88200283, -0.22957745, -0.11355825, ..., -0.6474616 ,
          0.5764707 ,  1.0511279 ],
        [ 0.30860043,  0.82822895, -0.8821481 , ...,  0.7463656 ,
         -1.0561934 , -0.14056098],
        [ 0.30860043,  0.82822895, -0.8821481 , ...,  0.7463656 ,
         -1.0561934 , -0.14056098],
        ...,
        [ 0.30860043,  0.82822895, -0.8821481 , ...,  0.7463656 ,
         -1.0561934 , -0.14056098],
        [ 0.30860043,  0.82822895, -0.8821481 , ...,  0.7463656 ,
         -1.0561934 , -0.14056098],
        [ 0.30860043,  0.82822895, -0.8821481 , ...,  0.7463656 ,
         -1.0561934 , -0.14056098]],

       [[ 0.73437667,  1.1547102 ,  0.76016843, ..., -1.0053995 ,
          0.89002705,  0.2620409 ],
        [ 1.0387673 , -0.27854785,  1.1961391 , ...,  1.0873531 ,
          1.4699728 ,  1.4143754 ],
        [ 0.08918951,  1.871771  , -0.8436212 , ...,  1.906895  ,
         -1.6093208 ,  0.05006287],
        .

In [52]:
class SequentialDecoder(tf.keras.layers.Layer):
    def __init__(self, layers):
        super().__init__()
        self.layers = layers

    def call(self, x,y,combined_mask,padding_mask):
        for layer in self.layers:
            x = layer(x,y,combined_mask,padding_mask)
        return x

In [61]:
class Decoder(tf.keras.layers.Layer):
    def __init__(self,d_model,num_heads,dff,num_decoders):
        super().__init__()
        self.num_decoders=num_decoders
        self.layers=SequentialDecoder([Decoder_layer(d_model=d_model,num_heads=num_heads,dff=dff) for _ in range(num_decoders)])
    def call(self,x,y,combined_mask,padding_mask):
        y=self.layers(x,y,combined_mask,padding_mask)
        return y
        

In [62]:
decoder=Decoder(d_model,num_heads,dff=512,num_decoders=12)
output=decoder(output,embedded_tokens,combined_mask,padding_mask)
output

<tf.Tensor: shape=(10000, 10, 128), dtype=float32, numpy=
array([[[-1.0006127 ,  0.18125199,  0.11331016, ..., -1.2837789 ,
          0.62591386,  0.9659846 ],
        [-1.4688057 ,  1.6708653 , -1.2456658 , ...,  0.9337999 ,
         -0.86432636, -0.4600804 ],
        [-1.4688057 ,  1.6708653 , -1.2456658 , ...,  0.9337999 ,
         -0.86432636, -0.4600804 ],
        ...,
        [-1.4688057 ,  1.6708653 , -1.2456658 , ...,  0.9337999 ,
         -0.86432636, -0.4600804 ],
        [-1.4688057 ,  1.6708653 , -1.2456658 , ...,  0.9337999 ,
         -0.86432636, -0.4600804 ],
        [-1.4688057 ,  1.6708653 , -1.2456658 , ...,  0.9337999 ,
         -0.86432636, -0.4600804 ]],

       [[-2.2500286 , -0.4298867 , -0.5748394 , ..., -1.4103285 ,
          0.01447373, -1.0572157 ],
        [-0.59413564, -0.15834834, -0.58492637, ...,  0.5862563 ,
         -0.48854172,  0.5284153 ],
        [-1.3027993 ,  1.5163426 , -2.3251445 , ...,  1.4927144 ,
         -2.2092578 , -1.8394185 ],
        .

In [63]:
len(decoder.trainable_variables)

312